## Obtenció de dades *Wikidata* amb SPARQL

Llibreries Python utilitzades:
* Pandas v1.5.0
* SPARQLWrapper v2.0.0







SPARQL online queries: https://query.wikidata.org/

### Configuració

* Imports
* Directori de dades

In [3]:
# Importem llibreires necessàries
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
from datetime import datetime, timedelta
import os
import time

# !pip install sparqlwrapper

# Directori per gruardar les dades
DATA = "data/wikipedia/"

### Functions

Definim les següients funcions:

* create_sparqlwrapper() => SPARQLWrapper()
* sparql_query(sparql, query) => time, results, sparql
* convert_to_dataframe(results, cols=None)

In [8]:
def create_sparqlwrapper():
    """Create SPARQLWrapper object"""
    
    
    WIKIDATA_ENDPOINT = "https://query.wikidata.org/sparql"
    SPARQL_USER_AGENT = 'nurasakiBot/0.0 (nurasaki@gmail.com)'
    
    # Create SPARQLWrapper
    sparql = SPARQLWrapper(WIKIDATA_ENDPOINT, agent=SPARQL_USER_AGENT)
    
    print("Creating SPARQLWrapper:")
    print("==="*15)
    print("Endpoint:", sparql.endpoint) 
    print("User-Agent:", sparql.agent) 
    
    return sparql



def sparql_query(sparql, query):
    """
    Executa query a: "https://query.wikidata.org/sparql
    Retorna: temps, resultats
    """
    
    init = datetime.now()

    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    # sparql.setMethod(method)

    # sparql.setTimeout(300)  # 5 minutes
    # https://python.hotexamples.com/examples/SPARQLWrapper/SPARQLWrapper/setTimeout/python-sparqlwrapper-settimeout-method-examples.html
    results = sparql.query().convert()
    
    return datetime.now()-init, results, sparql


def convert_to_dataframe(results, cols=None):
    "Parse SPARQL query results in JSON format to DataFrame"
    
    # cols = ['item','dateOfBirth', 'gender', 'profession']
    bindings = results['results']['bindings']    
    if len(bindings) > 0:
        if cols is None:
            cols = list(bindings[0].keys())
    
        df = pd.DataFrame([[r[col]['value'] for col in cols] for r in bindings], columns=cols)
    else:
        df = None    
    
    return len(bindings), df

### Get Wikidata persons with professions


In [13]:
DATA_SCRAP = DATA + "raw_persons_professions/"
if not os.path.exists(DATA_SCRAP):
    os.makedirs(DATA_SCRAP)

In [10]:
# Create SPARQLWrapper object
sparql = create_sparqlwrapper()

Creating SPARQLWrapper:
Endpoint: https://query.wikidata.org/sparql
User-Agent: nurasakiBot/0.0 (nurasaki@gmail.com)


#### Query

In [11]:
# Create raw SPARQL queries

# P31  property:  instance of
# Q5   class:     people
# P21  property:  gender
# P569 property:  date of birth
# P106 property:  occupation or profession
# Q28640 profession (Q28640) => occupation requiring specialized training

PREFIX = """
PREFIX bd: <http://www.bigdata.com/rdf#>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX wikibase: <http://wikiba.se/ontology#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
"""

select_str = "SELECT ?person ?dateOfBirth ?gender ?profession"
where_str = """
WHERE {{
    ?person wdt:P31 wd:Q5;
            wdt:P21 ?gender;
            wdt:P106 ?profession;
            wdt:P569 ?dateOfBirth. hint:Prior hint:rangeSafe true.
    {}
}}
"""

# Query optimitzation:
# https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/query_optimization#Optimization_strategies
# => wdt:P569 ?dateOfBirth. hint:Prior hint:rangeSafe true.

# => FILTER(YEAR(?dateOfBirth) = 1978) => no és tan eficient
# => FILTER("1978-00-00"^^xsd:dateTime <= ?dateOfBirth && ?dateOfBirth < "1979-00-00"^^xsd:dateTime)
filter_raw = 'FILTER("{}-00-00"^^xsd:dateTime <= ?dateOfBirth && ?dateOfBirth < "{}-00-00"^^xsd:dateTime)'

#### Loop years
    

In [24]:
print(f"{'time':<12}{'len_df':>10}{'years':>10}")
years_step = 5

for year in range(1900, 2010, years_step):
    
    # Create filter string (with years) and query
    filter_str = filter_raw.format(year, year+years_step)
    query = f"{PREFIX}{select_str}{where_str.format(filter_str)}"

    # Make query
    timedelta, results, sparql = sparql_query(sparql, query)
    bindings, df = convert_to_dataframe(results)

    # Agrupem ja que algunes referències tenen més d'una 'dateOfBirth' (depèn de la font)
    df = df.groupby(['person', 'gender', 'profession']).first().reset_index()
      
    # Save results DataFrame to DATA folder
    file = DATA_SCRAP + f"wiki_occupations_{year}.csv"
    if df is not None:
        df.to_csv(file, index=False)
    

    print(f"{str(timedelta)[:-3]:<12}{len(df):>10}  {year}-{year+years_step}")
    
    # Sleep 20 seconds to prevent query-limits
    time.sleep(20)

time            len_df     years
0:00:15.860     162891  1900-1905
0:00:27.568     153439  1905-1910
0:00:15.696     159626  1910-1915
0:00:05.265     142406  1915-1920
0:00:09.385     195338  1920-1925
0:00:10.649     211507  1925-1930
0:00:06.265     214035  1930-1935
0:00:06.300     234945  1935-1940
0:00:07.145     277282  1940-1945
0:00:09.375     329757  1945-1950
0:00:11.790     367323  1950-1955
0:00:08.431     341862  1955-1960
0:00:21.869     348138  1960-1965
0:00:18.081     321332  1965-1970
0:00:17.835     312860  1970-1975
0:00:07.349     290884  1975-1980
0:00:26.407     266487  1980-1985
0:00:09.605     237564  1985-1990
0:00:07.899     193533  1990-1995
0:00:06.927     133818  1995-2000
0:00:04.613     109072  2000-2005
0:00:01.630       7040  2005-2010


#### Merge years and separate genders

* male_id   = "Q6581097"
* female_id = "Q6581072"

In [27]:
# Concatenate all collected files in DATA folder
# ========================================================================================
df_years = pd.DataFrame()
for file in os.listdir(DATA_SCRAP):
    df_years = pd.concat([df_years, pd.read_csv(DATA_SCRAP+file)])
    
# Group by "profession", "gender"
cols = ["profession", "gender", "person"]
df_professions = df_years[cols].groupby(cols[:-1]).count().reset_index()

ENTITY_ENDPOINT = 'http://www.wikidata.org/entity/'
for col in cols[:-1]:
    df_professions[col] = df_professions[col].str.replace(ENTITY_ENDPOINT, "", regex=False)


# Filter "male" and "female"
# ========================================================================================
male_id, female_id = "Q6581097", "Q6581072"
df_female = df_professions[df_professions.gender == female_id]
df_male = df_professions[df_professions.gender == male_id]


# Clean data
# ========================================================================================
# Remove unknown professions
unk_endpoint = "http://www.wikidata.org/.well-known/"
df_male = df_male[~df_male.profession.str.startswith(unk_endpoint)]
df_female = df_female[~df_female.profession.str.startswith(unk_endpoint)]

# Rename columns 
df_male = df_male[["profession", "person"]].rename(columns={"person": "male"})
df_female = df_female[["profession", "person"]].rename(columns={"person": "female"})

# Merge male/female datafraes
df_profs = df_female.merge(df_male, left_on="profession", right_on="profession", how="outer").fillna(0)

# Convert dtypes to int
for col in ['female', 'male']:
    df_profs[col] = df_profs[col].astype('int')

# Compute totals and female ratios
df_profs["total"] = df_profs["female"] + df_profs["male"]
df_profs["female_ratio"] = df_profs["female"] / df_profs["total"]


# Save to CSV file
# ========================================================================================
df_profs.to_csv(DATA + 'wiki_occupations_by_gender_totals_1900-2020.csv', index=False)

### Get occupation labels

#### Read CSV file

In [28]:
# Read base CSV ocuupation file without 
profs_file = DATA + 'wiki_occupations_by_gender_totals_1900-2020.csv'

if not os.path.exists(profs_file):
    print("L'arxiu no existeix!")
    
# Create DataFrame
df_profs = pd.read_csv(profs_file)

#### Create SPARQLWrapper

In [29]:
sparql = create_sparqlwrapper()

Creating SPARQLWrapper:
Endpoint: https://query.wikidata.org/sparql
User-Agent: nurasakiBot/0.0 (nurasaki@gmail.com)


In [31]:
def parse_results(results, lang):
    """Parses SPARQL query of profession labels results to DataFrame."""
    
    rows = []
    for r in results['results']['bindings']:
        rows.append([
            r['occupation']['value'].replace(ENTITY_ENDPOINT ,""), 
            r['occupationLabel']['value'], 
            r.get('occupationAltLabel', {'value':""})['value']
        ])
    
    # Create DataFrame
    df = pd.DataFrame(rows, columns=["profession", f'label_{lang}', f'altLabel_{lang}'])
    
    # Remove labels with occupation ID
    df.loc[(df['profession']==df[f'label_{lang}']), f'label_{lang}'] = ""

    return df.groupby('profession').first().reset_index()

In [32]:
ENTITY_ENDPOINT = 'http://www.wikidata.org/entity/'
query_base = PREFIX + """
SELECT ?occupation ?occupationLabel ?occupationAltLabel
WHERE {{
  {{?occupation wdt:P31 wd:Q28640 .}}
    UNION
  {{?occupation wdt:P31/wdt:P279* wd:Q28640 .}}
  SERVICE wikibase:label {{ bd:serviceParam wikibase:language "{}" . }}
}}
"""
# Define base SPARQL query, where occupation entities and language are variables
# wdt:P31 => Instance of
# wdt:P279 => subclass of
# wd:Q28640 => Profession


# Creat DataFrame amb els resultats agrupats
df_labels = pd.DataFrame()

# Get Catalan Labels
time, results, _ = sparql_query(sparql, query_base.format("ca"))
print("Catalan labels, time:", time)
df_labels = parse_results(results, "ca")

# Get English Labels and merge results
time, results, _ = sparql_query(sparql, query_base.format("en"))
print("English labels, time:", time)
df_labels = df_labels.merge(parse_results(results, "en"))

Catalan labels, time: 0:00:23.533303
English labels, time: 0:00:21.696315


In [34]:
df_profs = df_profs.merge(df_labels, how="left").fillna("")
df_profs.to_excel("data/wikipedia/wiki_top_10500_professions-raw.xlsx", index=False)